In [2]:
import arxiv 
import requests
import os 
import re 
import urllib.request
import json
import re
import requests 
import os 



def standard_url_alphanumeric(input_string):
    allowed_characters = r'[a-zA-Z0-9\-._~:/?#\[\]@!$&\'()*+,;=%]'
    cleaned_string = re.sub(r'[^\w\s-]', ' ', input_string) 
    cleaned_string = re.sub(r'[-\s]+', '-', cleaned_string)
    cleaned_string = cleaned_string.strip("-")
    cleaned_string = "".join(re.findall(allowed_characters, cleaned_string))
    cleaned_string = cleaned_string.lower()
    return cleaned_string

def standard_url_alphanumeric_arxiv(input_string):
    return standard_url_alphanumeric(input_string).strip('-')

def standard_title_alphanumeric_chemarxiv(input_string):
    return standard_url_alphanumeric(input_string).strip('_')


In [ ]:
class Publication:
    def __init__(self,publication_filled, llm_use:bool=True) -> None:
        self.publication_filled = publication_filled
        self.title = self.get_publication_title()
        self.pdf = self.get_pdf()

    def get_publication_title(self) -> str:
        try:
            return self.publication_filled['bib']['title']
        except:
            return None
        
        
        
        
    def arxiv_downloader(self, directory):
        search = arxiv.Search(
            query = self.title, 
            max_results = 1, 
            sort_by = arxiv.SortCriterion.Relevance
        )
        client = arxiv.Client()
        results = client.results(search)
        for result in results:
            if result.title.lower() == self.title.lower():
                print("Article found.")
                url = result.pdf_url 
                self.pdf = url # Maybe to Split the function in 2
                
                
                
                response = requests.get(url)
                os.makedirs(directory, exist_ok = True)
                title = standard_url_alphanumeric_arxiv(self.title)
                with open(f"{title}.pdf", "wb") as f:
                    f.write(response.content)





    def search_chemrxiv_by_title(self, max_results: int = 1):
        search_query = self.title.split()
        chemrxiv_url = f'https://chemrxiv.org/engage/chemrxiv/public-api/v1/items?term={"%20".join(search_query)}&sort=RELEVANT_DESC&limit={max_results}'

        try:
            req = urllib.request.Request(
                url=chemrxiv_url, 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            with urllib.request.urlopen(req) as response:
                s = response.read()
                jsonResponse = json.loads(s.decode('utf-8'))
                dois = [item["item"]["doi"] for item in jsonResponse["itemHits"]]
                urls = ["https://doi.org/" + doi for doi in dois]
                ids = [item["item"]["id"] for item in jsonResponse["itemHits"]]
                titles = [item["item"]["title"].replace("\n", "") for item in jsonResponse["itemHits"]]
                titles = [standard_url_alphanumeric(self.title) for title in titles]
                pdfs = ["https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/" + id + "/original/" for id in ids]
                pdfs = [pdf + title + ".pdf" for pdf, title in zip(pdfs, titles)]
                return titles, pdfs

        except Exception as e:
            return f"An error occurred."

    def pdf_downloader(self, pdf_link, directory):
        os.makedirs(directory, exist_ok = True)
        
        try:
            response = requests.get(pdf_link)
            response.raise_for_status()
            title = standard_title_alphanumeric_chemarxiv(self.title)
            with open(f"{title}.pdf", "wb") as pdf_file:
                pdf_file.write(response.content)

        except Exception as e:
            print(f"Failed to donwload {pdf_link}")

